# Variant file

In [1]:
from pprint import pprint
import pandas as pd
from pysam import VariantFile

In [2]:
variants = VariantFile('data/snv-Somatic_all-972_0.vcf')

In [3]:
annotation_columns = variants.header \
    .info['ANN'] \
    .description \
    .split(':')[1] \
    .replace("'", "") \
    .split('|')
annotation_columns = tuple(map(lambda x: x.strip(), annotation_columns))

In [4]:
record = next(variants.fetch())
df = pd.DataFrame(columns=annotation_columns)
for annotation in record.info['ANN']:
    key_value_pairs = zip(annotation_columns, annotation.split('|'))
    df = df.append(dict(key_value_pairs), ignore_index=True)
df

,Allele,Annotation,Annotation_Impact,Gene_Name,Gene_ID,Feature_Type,Feature_ID,Transcript_BioType,Rank,HGVS.c,HGVS.p,cDNA.pos / cDNA.length,CDS.pos / CDS.length,AA.pos / AA.length,Distance,ERRORS / WARNINGS / INFO
0,A,stop_gained,HIGH,TP53,ENSG00000141510,transcript,ENST00000269305.8,protein_coding,8/11,c.892G>T,p.Glu298*,1082/2579,892/1182,298/393,,
1,A,stop_gained,HIGH,TP53,ENSG00000141510,transcript,ENST00000420246.6,protein_coding,8/12,c.892G>T,p.Glu298*,1025/2653,892/1026,298/341,,
2,A,stop_gained,HIGH,TP53,ENSG00000141510,transcript,ENST00000455263.6,protein_coding,8/12,c.892G>T,p.Glu298*,1025/2580,892/1041,298/346,,
3,A,stop_gained,HIGH,TP53,ENSG00000141510,transcript,ENST00000615910.4,protein_coding,8/11,c.859G>T,p.Glu287*,859/1149,859/1149,287/382,,


In [5]:
for variant in variants.fetch():
    print(variant.contig, variant.start, variant.stop)

chr17 7673727 7673728
chr17 58357955 58357956
chr4 54274526 54274527
chr9 95459699 95459700


# Reading BAM file

In [6]:
import pysam

bam = pysam.AlignmentFile('data/Deduped-972_0_sorted.bam')

In [7]:
bam.check_index()

True

In [23]:
base_counts = {
    'A': [],
    'C': [],
    'T': [],
    'G': [],
}
chromosome = 'chr17'
start_pos = 7673727
stop_pos = start_pos + 1
for pile in bam.pileup(contig=chromosome, start=start_pos):
    if pile.pos != start_pos:
        continue
    print ("\ncoverage at base %s = %s" % (pile.pos, pile.n))
    break
#     for read in pile.pileups:
#         base = read.alignment.query_sequence[read.query_position]
#         read_length = read.alignment.query_length
#         print(read_length)
#         base_counts[base].append(read_length)


coverage at base 7673727 = 7239


In [24]:
dir(pile)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'get_mapping_qualities',
 'get_num_aligned',
 'get_query_names',
 'get_query_positions',
 'get_query_qualities',
 'get_query_sequences',
 'n',
 'nsegments',
 'pileups',
 'pos',
 'reference_id',
 'reference_name',
 'reference_pos',
 'set_min_base_quality',
 'tid']

In [ ]:
pile.pileups

In [12]:
reads = [read for read in bam.fetch(contig=chromosome, start=start_pos)]

In [15]:
print(reads[100].infer_read_length())

96


In [ ]:
base_counts['G']

In [ ]:
sns.distplot(base_counts['A'])